In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f
!pip install pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-f option requires 1 argument
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
cd gdrive/MyDrive/Colab Notebooks

/content/gdrive/MyDrive/Colab Notebooks


In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-i127f7md/kobert-tokenizer_236105be8ea94da1b8ea80b652ed1b54
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-i127f7md/kobert-tokenizer_236105be8ea94da1b8ea80b652ed1b54


In [6]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-etxmeev6
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-etxmeev6


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained("skt/kobert-base-v1")

In [9]:
device = torch.device("cuda:0")

In [10]:
bertmodel, vocab = get_pytorch_kobert_model()
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

using cached model. /content/gdrive/MyDrive/Colab Notebooks/.cache/kobert_v1.zip
using cached model. /content/gdrive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
data = pd.read_csv('plz.csv', encoding='cp949')

In [12]:
data

,Q,A
0,"원고는 원고, E, G, H가 있는 페이스북 대화방에 이 사건 노출사진을 업로드...",강요
1,또한 원고는 기 중간 놀이 시간에 D가 같이 놀지 않겠다고 하자 D를 때리려고 하였다,강요
2,"그것까지는 못한다고 거절했더니 ""알았다""고 했지만 알았다는 말이 우리 관계를 소문...",강요
3,원고는 카카오톡 메신저로 피해학생에게 2018년 유튜브 사건과 관련한 3천자 사...,강요
4,"""라는 내용의 본인확인서를 작성하도록 강요하였으며, ③ 원고 문○○이 자퇴서를 작...",강요
...,...,...
1065,N 중학교 시절부터 잘못했을 때는 무릎을 꿇고 맞아서 당연한 것이라고 생각하였다,해당없음
1066,그래서 이번 일도 큰 일이 아니라고 생각하였다,해당없음
1067,원고 D,해당없음
1068,( 8,해당없음


In [13]:
data.loc[(data['A'] == "강요"), 'A'] = 0
data.loc[(data['A'] == "금품갈취"), 'A'] = 1
data.loc[(data['A'] == "따돌림"), 'A'] = 2
data.loc[(data['A'] == "사이버폭력"), 'A'] = 3
data.loc[(data['A'] == "성폭력"), 'A'] = 4
data.loc[(data['A'] == "신체폭력"), 'A'] = 5
data.loc[(data['A'] == "언어폭력"), 'A'] = 6
data.loc[(data['A'] == "협박"), 'A'] = 7
data.loc[(data['A'] == "해당없음"), 'A'] = 8



data_list = []
for ques, label in zip(data['Q'], data['A'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [15]:
!pip install --no-cache-dir transformers sentencepiece 

from transformers import AutoTokenizer, AutoModelForMaskedLM

# kcbert의 tokenizer와 모델을 불러옴.
kcbert_tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
kcbert = AutoModelForMaskedLM.from_pretrained("beomi/kcbert-base")

result = kcbert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
print(kcbert_tokenizer.vocab['대선'])
print([kcbert_tokenizer.encode(token) for token in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['너는', '내년', '대선', '때', '투표', '##할', '수', '있어', '?']
9311
[[2, 9039, 3], [2, 8946, 3], [2, 9311, 3], [2, 1003, 3], [2, 8701, 3], [2, 7, 7, 3358, 3], [2, 1931, 3], [2, 8749, 3], [2, 32, 3]]


In [16]:
!pip install --no-cache-dir transformers sentencepiece

from transformers import AutoModel, AutoTokenizer

kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
kobert = AutoModel.from_pretrained("skt/kobert-base-v1")

result = kobert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
kobert_vocab = kobert_tokenizer.get_vocab()
print(kobert_vocab.get('▁대선'))
print([kobert_tokenizer.encode(token) for token in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['▁너', '는', '▁내년', '▁대선', '▁때', '▁투표', '할', '▁수', '▁있어', '?']
1654
[[1457, 3, 2], [517, 5760, 3, 2], [1437, 3, 2], [1654, 3, 2], [1844, 3, 2], [4772, 3, 2], [4977, 3, 2], [2872, 3, 2], [3868, 3, 2], [633, 3, 2]]


In [17]:
# Setting parameters
max_len = 128
batch_size = 32
warmup_ratio = 0.1
num_epochs = 20  
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [18]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [19]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [20]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [23]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/27 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.137909412384033 train acc 0.1875
epoch 1 train acc 0.17901234567901234


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 19 test acc 0.6103896103896104


  0%|          | 0/27 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.3760043680667877 train acc 0.96875
epoch 20 train acc 0.8503086419753086


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 20 test acc 0.6103896103896104


In [24]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("강요")
            elif np.argmax(logits) == 1:
                test_eval.append("금품갈취")
            elif np.argmax(logits) == 2:
                test_eval.append("따돌림")
            elif np.argmax(logits) == 3:
                test_eval.append("사이버폭력")
            elif np.argmax(logits) == 4:
                test_eval.append("성폭력")
            elif np.argmax(logits) == 5:
                test_eval.append("신체폭력")
            elif np.argmax(logits) == 6:
                test_eval.append("언어폭력")
            elif np.argmax(logits) == 7:
                test_eval.append("협박")
            elif np.argmax(logits) == 8:
                test_eval.append("해당없음")      

        print(">>" + test_eval[0] + " 입니다.")

In [25]:
data

['자 학교폭력 사안조사 보고서', '8']

In [ ]:
쉬\end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 담배빵을 맞았어요


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>>신체폭력 입니다.


하고싶은 말을 입력해주세요 : 나를 비꼬았어요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 이번주에 해야할 거를 어제 다 했다고 거짓말을 쳐요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 욕을 이제 안한다고 거짓말을 쳐요
>>언어폭력 입니다.


하고싶은 말을 입력해주세요 : 뒤에서 자꾸 욕해요
>>언어폭력 입니다.


하고싶은 말을 입력해주세요 : 매주 개인면담을 하자고 해요
>>강요 입니다.


하고싶은 말을 입력해주세요 : 매주 개인면담 하자고 불러요
>>강요 입니다.


하고싶은 말을 입력해주세요 : 매주 개인면담을 하자해요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 매주 술 마시자해요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 강제로 술을 따라요
>>신체폭력 입니다.


하고싶은 말을 입력해주세요 : 윤영철
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 김민지
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 맨날 연락이와요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 자주 연락을 해요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 아무이유없이 연락해요
>>따돌림 입니다.


하고싶은 말을 입력해주세요 : 저에게 자꾸 말을 시켜요
>>언어폭력 입니다.


하고싶은 말을 입력해주세요 : 커피를 사오라했어요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 돈을 뺏겼어요
>>신체폭력 입니다.


하고싶은 말을 입력해주세요 : 지갑을 뺏었어요
>>신체폭력 입니다.


하고싶은 말을 입력해주세요 : 뺏겼어요
>>언어폭력 입니다.


하고싶은 말을 입력해주세요 : 바나프레소에서 2,500원 커피를 사오라했어요
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 2500원
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 17,000원
>>해당없음 입니다.


하고싶은 말을 입력해주세요 : 74,000원
>>해당없음 입니다.


하고싶은 말을 